In [16]:
import pandas as pd
from dfply import *
import glob
from sklearn import preprocessing

In [2]:
headerlist = ['timestamp']
for i in range(1,6):
    for j in range(1,257):
        headerlist.append('imf_'+str(i)+'_feature_'+str(j))
files = []
for filepath in glob.iglob(r"E:\\ProjectData\\ERAU-REU\\Project-Drone-Behavior\\behavior-captures\\*.csv"):
    files.append(filepath)

for file in files:
    df = pd.read_csv(file,header=None)
    df.columns = headerlist
    df.to_csv(file,index=False)

In [5]:
#creating a labelled and scaled dataset for just the flight behavior
files = []
for filepath in glob.iglob(r"E:\\ProjectData\\ERAU-REU\\Project-Drone-Behavior\\behavior-captures\\*.csv"):
    files.append(filepath)
#feature engineering a new column to act as the output for predictions. This will be the behavior that we want to predict on
first = True
processed = False
for filepath in files:
    processed = False
    if 'processed' in filepath:
        processed = True
        pass
    elif 'straight' in filepath:
        df = pd.read_csv(filepath, skiprows=1)
        df.drop('timestamp',axis=1,inplace=True)
        df['behavior'] = 'straight'
    elif 'surround' in filepath:
        df = pd.read_csv(filepath, skiprows=1)
        df.drop('timestamp',axis=1,inplace=True)
        df['behavior'] = 'surround'
    else:
        df = pd.read_csv(filepath,skiprows=1)
        df.drop('timestamp',axis=1,inplace=True)
        df['behavior'] = 'noise'
    
    #create new dataframe and append all to it
    if processed:
        pass
    elif first and not processed:
        newDf = df
        first = False
    else:
        newDf = newDf.append(df,ignore_index=True)
#normalizing the data by center and scale
#causes all of the datapoints to be between 1 and 0 based on their z-score
valuesDf = newDf.drop('behavior',axis=1)
x = valuesDf.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled,columns=valuesDf.columns,index=newDf.index)
df['behavior'] = newDf['behavior']
df.drop_duplicates(inplace=True)
df.to_csv("E:\\ProjectData\\ERAU-REU\\Project-Drone-Behavior\\behavior-captures\\behavior-processed.csv",index=False)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Columns: 1280 entries, imf_1_feature_1 to imf_5_feature_256
dtypes: float64(1280)
memory usage: 24.4 MB
[[4.52254151e-03 9.30378554e-03 3.32687991e-03 ... 5.49827679e-05
  5.49335572e-05 5.49113022e-05]
 [1.78005728e-03 1.37523978e-03 1.55987688e-03 ... 5.11882669e-05
  5.11754800e-05 5.11774442e-05]
 [9.86228682e-03 2.54181029e-03 1.21646285e-03 ... 4.03334816e-05
  4.03154298e-05 4.02704545e-05]
 ...
 [4.40789560e-04 1.15648357e-03 2.83372246e-03 ... 1.45032554e-03
  2.83372246e-03 1.15648357e-03]
 [9.40556112e-04 8.95314646e-04 2.76864460e-03 ... 1.00687515e-03
  2.76864460e-03 8.95314646e-04]
 [1.15255951e-04 4.79831821e-04 1.33876428e-03 ... 1.54224722e-03
  1.33876428e-03 4.79831821e-04]]


In [24]:
#creating a labelled and scaled dataset for the additional features of the dataset as well. 
files = []
for filepath in glob.iglob(r"E:\\ProjectData\\ERAU-REU\\Project-Drone-Behavior\\behavior-captures\\*.csv"):
    files.append(filepath)
#feature engineering a new column to act as the output for predictions. This will be the behavior that we want to predict on
first = True
processed = False
for filepath in files:
    first_char = filepath[69].lower()
    processed = False
    if 'processed' in filepath:
        processed = True
        pass
    elif 'straight' in filepath and first_char == 'c':
        df = pd.read_csv(filepath,skiprows=1)
        df.drop('timestamp',axis=1,inplace=True)
        df['behavior'] = 'straight'
        df['signal'] = 'controller'
    elif 'straight' in filepath and first_char == 'v':
        df = pd.read_csv(filepath,skiprows=1)
        df.drop('timestamp',axis=1,inplace=True)
        df['behavior'] = 'straight'
        df['signal'] = 'video'
    elif 'surround' in filepath and first_char == 'c':
        df = pd.read_csv(filepath,skiprows=1)
        df.drop('timestamp',axis=1,inplace=True)
        df['behavior'] = 'surround'
        df['signal'] = 'controller'
    elif 'surround' in filepath and first_char == 'v':
        df = pd.read_csv(filepath,skiprows=1)
        df.drop('timestamp',axis=1,inplace=True)
        df['behavior'] = 'surround'
        df['signal'] = 'video'
    else:
        df = pd.read_csv(filepath,skiprows=1)
        df.drop('timestamp',axis=1,inplace=True)
        df['behavior'] = 'noise'
        df['signal'] = 'none'
    if processed:
        pass
    elif 'green' in filepath and 'yellow' in filepath:
        df['multiple'] = 1
    else:
        df['multiple'] = 0 
    #create new dataframe and append all to it
    if processed:
        pass
    elif first and not processed:
        newDf = df
        first = False
    else:
        newDf = newDf.append(df,ignore_index=True)
#normalizing the data by center and scale
#causes all of the datapoints to be between 1 and 0 based on their z-score
valuesDf = newDf.drop(['behavior','signal','multiple'],axis=1)
x = valuesDf.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled,columns=valuesDf.columns,index=newDf.index)
df['behavior'] = newDf['behavior']
df['signal'] = newDf['signal']
df['multiple'] = newDf['multiple']
df.drop_duplicates(inplace=True)
 
df.to_csv("E:\\ProjectData\\ERAU-REU\\Project-Drone-Behavior\\behavior-captures\\behavior-signal-multiple-processed.csv",index=False)
